In [5]:
"""
Input format for internalwave_analyzer
Year 	 Month 	 Day 	 Hour 	 Minute 	 Wind 	 Direction  Solar Rad.
2015	6	17	7	30	1.77778	247.5   6.287
2015	6	17	8	0	2.22222	202.5   13.14
2015	6	17	8	30	1.33333	225 7.5
2015	6	17	9	0	1.77778	225 10.3
"""

'\nInput format for internalwave_analyzer\nYear \t Month \t Day \t Hour \t Minute \t Wind \t Direction  Solar Rad.\n2015\t6\t17\t7\t30\t1.77778\t247.5   6.287\n2015\t6\t17\t8\t0\t2.22222\t202.5   13.14\n2015\t6\t17\t8\t30\t1.33333\t225 7.5\n2015\t6\t17\t9\t0\t1.77778\t225 10.3\n'

"""
Meteo data from alplakes
{
  "time":["2023-01-01T00:00:00+00:00",
    "2023-01-01T01:00:00+00:00"],
  "variables": {
    "vapour_pressure": {
      "unit": "hPa",
      "description": "Vapour pressure 2 m above ground",
      "data": [
        11,
        11,
        10.9,
        10.9]},
    "global_radiation": {
      "unit": "W/m²",
      "description": "Global radiation",
      "data": [
        3,
        3]},
    "air_temperature": {
      "unit": "°C",
      "description": "Air temperature 2 m above ground",
      "data": [
        12.1,
        12.1,
        11.6]},
    "precipitation": {
      "unit": "mm",
      "description": "Precipitation",
      "data": [
        0,
        0,
        0,
        0]},
    "wind_speed": {
      "unit": "m/s",
      "description": "Wind speed scalar",
      "data": [
        1.9,
        2,
        2.1]},
    "wind_direction": {
      "unit": "°",
      "description": "Wind direction",
      "data": [
        310,
        324,
        355]
    }
}
"""

In [11]:
import pandas as pd
import numpy as np
from datetime import datetime
import xarray as xr
import json

In [42]:
input_folder_name = "lexplore_summer_2021"
date_start = pd.Timestamp("2021-06-01").tz_localize('UTC')
date_end = pd.Timestamp("2021-08-01").tz_localize('UTC')

input_json_name = "meteo_pully_2021"

input_json_path = rf"..//runs//{input_folder_name}//raw_input//{input_json_name}.json"
output_path = rf"..//runs//{input_folder_name}//{input_json_name}.met"



# Load data

In [17]:
with open(input_json_path) as json_file:
    input_data = json.load(json_file)

In [27]:
input_data['variables'].keys()

dict_keys(['vapour_pressure', 'global_radiation', 'air_temperature', 'precipitation', 'wind_speed', 'wind_direction'])

# Convert Json to Dataframe

In [38]:
# Extract data
times = input_data["time"]
wind_speeds = input_data['variables']["wind_speed"]['data']
wind_directions = input_data['variables']["wind_direction"]['data']
solar_radiation = input_data['variables']["global_radiation"]['data']

# Build a list of rows
rows = []
for t, ws, wd, sr in zip(times, wind_speeds, wind_directions, solar_radiation):
    dt = datetime.fromisoformat(str(t))
    rows.append([
        dt.year,
        dt.month,
        dt.day,
        dt.hour,
        dt.minute,
        ws,
        wd,
        sr,
        dt
    ])

# Create DataFrame
df = pd.DataFrame(rows, columns=["Year", "Month", "Day", "Hour", "Minute", "Wind", "Direction", "Solar Rad.", "time"])

# Filter by date

In [39]:
filtered_df = df[(df['time'] > date_start)&(df['time'] < date_end)]

In [40]:
filtered_df

,Year,Month,Day,Hour,Minute,Wind,Direction,Solar Rad.,time
3625,2021,6,1,1,0,2.1,29,3,2021-06-01 01:00:00+00:00
3626,2021,6,1,2,0,2.3,29,2,2021-06-01 02:00:00+00:00
3627,2021,6,1,3,0,2.1,25,3,2021-06-01 03:00:00+00:00
3628,2021,6,1,4,0,2.1,23,4,2021-06-01 04:00:00+00:00
3629,2021,6,1,5,0,1.8,24,37,2021-06-01 05:00:00+00:00
...,...,...,...,...,...,...,...,...,...
5083,2021,7,31,19,0,2.2,322,5,2021-07-31 19:00:00+00:00
5084,2021,7,31,20,0,1.1,124,2,2021-07-31 20:00:00+00:00
5085,2021,7,31,21,0,0.8,27,2,2021-07-31 21:00:00+00:00
5086,2021,7,31,22,0,0.9,360,3,2021-07-31 22:00:00+00:00


# Save

In [43]:
filtered_df.drop("time", axis=1).to_csv(output_path, sep="\t", index=False)
print(output_path)

..//runs//lexplore_summer_2021//meteo_pully_2021.met
